<a href="https://colab.research.google.com/github/vindruid/image_quality/blob/master/devina_cloths.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Change here, fill with your google drive folder structure
import os

folder_ori    = os.path.join("devina", "original")
folder_rm     = os.path.join("devina", "removed")
folder_result = os.path.join("devina", "result")

In [0]:
import numpy as np

import cv2

import matplotlib.pyplot as plt
from tqdm import tqdm

from google.colab import drive

def resize_transparant(path_image_ori, path_image_rm):
  removed = plt.imread(path_image_rm)
  potrait = removed.shape[0] > removed.shape[1] # H > W
  if potrait:
    resized = cv2.resize(removed, (2592, 4608), interpolation = cv2.INTER_AREA) #w,h
  else: 
    resized = cv2.resize(removed, (4608, 2592), interpolation = cv2.INTER_AREA) #w,h

  ori = plt.imread(path_image_ori)
  ori = cv2.cvtColor(ori, cv2.COLOR_BGR2BGRA)
  potrait_ori = ori.shape[0] > ori.shape[1] # H > W

  if not potrait == potrait_ori: # one potrait, one landscape
    ori = cv2.rotate(ori, cv2.ROTATE_90_CLOCKWISE)

  ori_removed = ori.copy()
  ori_removed = ori_removed / 255.
  ori_removed[:,:,3] = resized[:,:,3]

  return ori_removed

path_drive = os.path.join("drive", "My Drive")

drive.mount('/content/drive')


In [0]:
# Define paths
path_ori = os.path.join(path_drive, folder_ori)
path_rm = os.path.join(path_drive, folder_rm)
path_result = os.path.join(path_drive, folder_result)
if not os.path.exists(path_result):
  os.mkdir(path_result)
  
# PROCESS the photos 
list_filename_ori = sorted(os.listdir(path_ori))
for filename_ori in tqdm(list_filename_ori):
  file_id = filename_ori.split('.')[0]
  filename_rm = f'{file_id}-removebg-preview.png'
  filename_res = f'{file_id}_result.png'

  path_image_ori = os.path.join(path_ori, filename_ori)
  path_image_rm  = os.path.join(path_rm, filename_rm)
  path_image_res = os.path.join(path_result, filename_res)

  if not os.path.exists(path_image_rm): 
    print(f"{filename_rm} IS NOT FOUND")
    continue

  ori_removed = resize_transparant(path_image_ori, path_image_rm)

  plt.imsave(path_image_res, ori_removed)